#### Import packages

In [1]:
import pandas as pd
import numpy as np
import pandas_access as mdb
import os
import sys
from datetime import datetime
import glob
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import geopandas as gpd
import itertools
import folium

from scipy.spatial import cKDTree

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 75
pd.options.display.max_rows = 250

#### Functions

In [2]:
def get_time():
    """
    Returns current timestamp.
    """
    return datetime.now().strftime("%m-%d-%Y %H:%M:%S")

#### Configuration

In [3]:
months = [202103, 202104]
routes = ['30N', '30S', '31', '33']
days = ['Tuesday']
directions = ['EAST', 'SOUTH']
start_time = '05:00:00'
end_time = '10:00:00'
CRS = 'EPSG:4326'
distance_thresh = 400
duration_thresh = 1.5

#### Directory

In [4]:
root_directory = os.path.normpath(r'C:\Users\amondal\OneDrive - Cambridge Systematics\CS PROJECTS\WMATA Bus Priority\data\tsp_exploratory')

path_repo = os.path.join(r'C:\Users\amondal\OneDrive - Cambridge Systematics\CS PROJECTS\WMATA Bus Priority\codebase\WMATA_AVL')
sys.path.append(path_repo)

# Import wmatarawnav library
import wmatarawnav as wr

#### Read rawnav data

In [5]:
def read_rawnav_parquet(root_directory, month, routes, days):
    return (wr.read_cleaned_rawnav(analysis_routes_= routes,
                                   analysis_days_ = days,
                                   path = os.path.join(root_directory, 'rawnav_data_wisconsin', f'rawnav_data_{month}.parquet')))

data = pd.concat([read_rawnav_parquet(root_directory, month, routes, days) for month in months])

# Sort data by `start_date_time`, `route`, and `index_loc`
data = data.sort_values(by = ['start_date_time', 'route', 'index_loc'])

# Apply filters | AM Peak (5 am - 10 am)
data = data.set_index('start_date_time').between_time(start_time, end_time, include_end = False).reset_index()

# Create unique trip ID
data['trip_id'] = data.index_run_start.astype(int).astype(str) + ' - ' + data.filename

#### Read and process schedule database

In [6]:
# Set up connection to WMATA Schedule database
# Create cursor object

schedule_filename = 'Schedule_082719-201718.mdb'
database_path = os.path.join(root_directory, 'wmata_schedule', schedule_filename)
connection = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + database_path)
cursor = connection.cursor()

In [7]:
# Read required tables

pattern = pd.read_sql(sql = 'SELECT * FROM Pattern', con = connection)
stop = pd.read_sql(sql = 'SELECT * FROM Stop', con = connection)
stop_info = pd.read_sql(sql = 'SELECT * FROM StopInfo', con = connection)
stop_list = pd.read_sql(sql = 'SELECT * FROM StopList', con = connection)

In [8]:
# Select required columns in each table

pattern = pattern[['PatternID', 'TARoute', 'PatternName', 'CDVariation', 'Direction', 'Distance', 'RouteKey']]
stop = stop[['GeoID', 'GeoDescription', 'Longitude', 'Latitude']]
stop_info = stop_info[['StopID', 'StopDesc']]
stop_list = stop_list[['RouteKey', 'StopSequence', 'StopID', 'GeoID']]

In [9]:
# Filter and merge datasets

pattern = pattern[pattern.TARoute.isin(routes)].sort_values(by = ['TARoute', 'CDVariation'])
stops = stop_list.merge(stop, on = 'GeoID', how = 'left')
stops = stops[stops.RouteKey.isin(pattern.RouteKey.unique())].sort_values(by = ['RouteKey', 'StopSequence'])
stops = pattern.merge(stops, on = 'RouteKey', how = 'right')

In [10]:
# Create route_pattern column to merge with rawnav
stops['route_pattern'] = stops.TARoute.astype(str) + stops.CDVariation.astype(str).str.zfill(2)

In [11]:
# Remove whitespaces from route_pattern columns in rawnav data
# Merge route pattern to get Direction

data['route_pattern'] = data['route_pattern'].str.strip()
data = data.merge(stops[['route_pattern', 'Distance', 'Direction']].drop_duplicates(keep = 'first'),
                  on = 'route_pattern',
                  how = 'left')

# Filter by direction only
data = data[data.Direction.isin(directions)]

In [15]:
data.groupby(['route_pattern'])['trip_id'].nunique()

route_pattern
30N02     45
30S02     46
3101     122
3301     159
Name: trip_id, dtype: int64

#### Import TSP Intersection Coordinates

In [16]:
# Import TSP intersection coordinates and convert to GeoDataFrame
tsp_int = (pd.read_excel(os.path.join(root_directory, 
                                     'tsp_intersections', 
                                     'wisconsin_tsp_intersections.xlsx'),
                       sheet_name = 'tsp_intersections')
           .rename(columns = {'Intersection ' : 'intersection', 'xcoor' : 'int_long', 'ycoor' : 'int_lat'})
           .dropna(subset = ['intersection']))[['intersection', 'int_lat', 'int_long']]

tsp_int = gpd.GeoDataFrame(tsp_int, 
                           geometry = gpd.points_from_xy(tsp_int.int_long, tsp_int.int_lat),
                           crs = CRS)

#### Plot Intersection Coordinates

In [ ]:
def plot_points_on_map(gdf, y_col, x_col, label, radius = 25, color = 'crimson'):
    
    map_with_points = folium.Map(location = [gdf[y_col].mean(), gdf[x_col].mean()],
                                 #tiles = "Stamen Toner", 
                                 zoom_start = 15)
    # Add points
    for ix, row in gdf.iterrows():
        folium.Circle(radius = radius,
                      location = [row[y_col], row[x_col]],
                      popup = row[label],
                      color = color,
                      fill = False).add_to(map_with_points)
    return map_with_points

In [ ]:
'''
plot_points_on_map(gdf = tsp_int, 
                   x_col = 'int_long', 
                   y_col = 'int_lat', 
                   label = 'intersection')
'''

#### Trip Filtering

In [ ]:
# Based on https://gis.stackexchange.com/questions/222315/geopandas-find-nearest-point-in-other-dataframe
def get_nearest_point(gdA, gdB):
    '''
    Find the nearest point from gdB for 
    each point in gdA.  
    '''
    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k = 1)
    gdB_nearest = gdB.iloc[idx].drop(columns = 'geometry').reset_index(drop = True)
    gdf = pd.concat([gdA.reset_index(drop=True), gdB_nearest, pd.Series(dist, name = 'dist')], axis=1)
    return gdf

In [ ]:
def dict2_to_df(dictionary, col_name_1, col_name_2, col_name_3):
    '''
    Creates a Pandas DataFrame from 
    2-layer nested dictionary.
    '''
    df = (pd.concat({k: pd.DataFrame.from_dict(v, orient = 'index') for k, v in dictionary.items()})
          .reset_index()
          .rename(columns = {'level_0' : col_name_1, 'level_1' : col_name_2, 0 : col_name_3}))
    return df

In [ ]:
def create_geo_dataframe(df, longitude_col, latitude_col, crs):
    '''
    Create a GeoDataFrame from a Pandas DataFrame
    with longitude and longitude of points.
    '''
    import geopandas as gpd
    return gpd.GeoDataFrame(df, 
                            geometry = gpd.points_from_xy(df[longitude_col], df[latitude_col]),
                            crs = CRS)

#### Create Stops and RAWNAV GeoDataFrame

In [ ]:
stops = create_geo_dataframe(df = stops, 
                             longitude_col = 'Longitude', 
                             latitude_col = 'Latitude', 
                             crs = CRS)

In [ ]:
data = create_geo_dataframe(df = data,
                            longitude_col = 'long',
                            latitude_col = 'lat',
                            crs = CRS)

#### Define Trip Start Point

In [ ]:
def remove_pings_before_first_stop_window(data, unique_trip_id_col, first_stop_entry_point = 'E00', first_stop_exit_point = 'X-1'):
    """
    Remove pings that are recorded before the exit point
    of the first stop window. This is done in multiple steps
    to improve computational efficiency.
    
    Steps:
    ------
    - Remove the trips that do not have first stop window entry tag.
    - Identify the entry point of first stop window.
    - Remove the records before the entry point of first stop window.
    - Identify the exit point of first stop window.
    - Remove the trips that do not have stop window exit tag.
    - Remove the records before the exit point of first stop window.
    """
    
    # Remove the trips that do not have first stop window entry tag.
    data = (data[data[unique_trip_id_col]
                 .isin(data[data.stop_window == first_stop_entry_point][unique_trip_id_col].unique())])
    
    # Identify the entry point of first stop window.
    df = (data[data.stop_window == first_stop_entry_point]
          .groupby([unique_trip_id_col, 'stop_window'])['index_loc']
          .agg(['min'])
          .add_prefix('fsw_entry_index_loc_')
          .sort_index()
          .reset_index())
    
    data = data.merge(df.drop(columns = ['stop_window']),
                      on = unique_trip_id_col,
                      how = 'left').sort_values(by = ['start_date_time', 'route', 'index_loc'])
    
    # Remove the records before the entry point of first stop window.
    data[data.index_loc >= data.fsw_entry_index_loc_min]
    
    # Identify the exit point of first stop window. 
    df2 = (data[data.stop_window == first_stop_exit_point]
            .groupby([unique_trip_id_col, 'stop_window'])['index_loc']
            .agg(['min'])
            .add_prefix('fsw_exit_index_loc_')
            .sort_index()
            .reset_index())
    
    # Remove the trips that do not have stop window exit tag.
    data = data[data[unique_trip_id_col].isin(df2[unique_trip_id_col].unique())]
    data = data.merge(df2.drop(columns = ['stop_window']),
                      on = unique_trip_id_col,
                      how = 'left').sort_values(by = ['start_date_time', 'route', 'index_loc'])
    
    # Remove the records before the exit point of first stop window.
    data = data[data.index_loc >= data.fsw_exit_index_loc_min]
    return data

In [ ]:
# Remove pings that are recorded before the entry point
# of the first stop window.
data = remove_pings_before_first_stop_window(data = data, 
                                             unique_trip_id_col = 'trip_id')

#### Define Trip End Point

In [ ]:
def remove_pings_after_last_stop_window(data, unique_trip_id_col):
    """
    Remove pings that are recorded after the entry point
    of the last stop window. This is done in multiple steps
    to improve computational efficiency.
    
    Steps:
    ------
    - Create dummy variable for stop window entry tag.
    - Identify the entry point of last stop window.
    - Remove the trips that do not have last stop window entry tag.
    - Remove the records after the entry point of last stop window.
    """
    
    # Create dummy variable for stop window entry tag.
    data.loc[data['stop_window'].str[0] == 'E', 'sw_entry_marker'] = 'E'
    
    # Identify the entry point of last stop window.
    df = (data.groupby([unique_trip_id_col, 'sw_entry_marker'])['index_loc']
          .agg(['max'])
          .add_prefix('lsw_entry_index_loc_')
          .sort_index()
          .reset_index())
    
    # Remove the trips that does not have last stop window entry tag.
    data = data[data[unique_trip_id_col].isin(df[unique_trip_id_col].unique())]
    data = data.merge(df.drop(columns = ['sw_entry_marker']),
                  on = unique_trip_id_col,
                  how = 'left').sort_values(by = ['start_date_time', 'route', 'index_loc'])
    
    # Remove the records after the entry point of last stop window.
    data = data[data.index_loc <= data.lsw_entry_index_loc_max]
    return data

In [ ]:
# Remove pings that are recorded after the entry point
# of the last stop window.
data = remove_pings_after_last_stop_window(data = data, unique_trip_id_col = 'trip_id')

#### Reset Odometer and Timer

In [ ]:
def reset_odometer_timer(data, unique_trip_id_col = 'trip_id'):
    """
    Reset odometer and timer reading based on the start 
    and end point of each trip. 
    """
    df = (data.groupby([unique_trip_id_col])['odom_ft'].agg(['min']).add_prefix('odom_ft_')
          .join(data.groupby([unique_trip_id_col])['sec_past_st'].agg(['min']).add_prefix('sec_past_st_')).reset_index())
    
    data = data.merge(df, on = unique_trip_id_col, how = 'left')
    
    # Create new columns with adjusted variables.
    data['adj_odom_ft'] = data['odom_ft'] - data['odom_ft_min']
    data['adj_dur_sec'] = data['sec_past_st'] - data['sec_past_st_min']
    return data

In [ ]:
# Reset odometer and timer
data = reset_odometer_timer(data = data, unique_trip_id_col = 'trip_id')

#### Select Trips based on Median Distance and Duration

In [ ]:
def select_trips_based_on_dist_dur(data, unique_trip_id_col, adj_odom_col, adj_dur_col, agg_func, distance_thresh, duration_thresh):
    """
    Select trips that have distance and duration within 
    certain range of aggregate values of those measures
    for each route-pattern combination.
    """
    trip_sel_df = (data.groupby(['route_pattern', unique_trip_id_col])[adj_odom_col, adj_dur_col].max()
                   .reset_index().set_index('route_pattern'))
    trip_sel_df = trip_sel_df.join(data.drop(columns = ['trip_id'])[['route_pattern', adj_odom_col, adj_dur_col]]
                                   .groupby(['route_pattern'])
                                   .quantile(0.99).add_prefix('agg_'))


    # Selection criteria
    select_criteria = ((trip_sel_df[adj_odom_col] > trip_sel_df[f'agg_{adj_odom_col}'] - distance_thresh) &
                       (trip_sel_df[adj_odom_col] < trip_sel_df[f'agg_{adj_odom_col}'] + distance_thresh) &
                       (trip_sel_df[adj_dur_col] < trip_sel_df[f'agg_{adj_dur_col}'] * duration_thresh))
    trip_sel_df['selection_flag'] = 1 * select_criteria
    
    return trip_sel_df

In [ ]:
# Please use this function with caution. This is not properly tested.
trip_sel_df = select_trips_based_on_dist_dur(data = data,
                                             unique_trip_id_col = 'trip_id',
                                             adj_odom_col = 'adj_odom_ft', 
                                             adj_dur_col = 'adj_dur_sec', 
                                             agg_func = np.median, 
                                             distance_thresh = distance_thresh, 
                                             duration_thresh = duration_thresh).reset_index()

In [ ]:
# Select trips
data = data[data.trip_id.isin(trip_sel_df[trip_sel_df.selection_flag == 1].trip_id.unique())]